<a href="https://colab.research.google.com/github/jasonjstewart/college-football/blob/master/EDA_College_Football_Field_Goals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This .ipynb goes through the process of adding or creating fields. Each block of code has a comment describing what that specific block of code is doing.  While, this may not be the most efficient way to do things it was effective. The original .csv file comes from a SQL Server query based on the data recieved from https://api.collegefootballdata.com/api/docs/?url=/api-docs.json#/.

In [13]:
#Import data and check for missing values or irregularities
import pandas as pd
df=pd.read_csv('https://jasonstewart.s3.amazonaws.com/college-football-data.csv')
df.drop(['Unnamed: 0','Unnamed: 0.1'],axis=1, inplace=True)
df.describe()

,id,offense_score,defense_score,period,clock,yards_to_goal,down,distance,fg_length,ppa,drive_start_yard_line,number_of_plays_in_drive,yards_gained_during_drive,season,fg_result,score_diff
count,3.279700e+04,32797.000000,32797.000000,32797.000000,32797.000000,32797.000000,32797.000000,32797.000000,32797.000000,32797.000000,32797.000000,32797.000000,32797.000000,32797.000000,32797.000000,32797.000000
mean,1.825968e+17,15.003384,11.870506,2.444888,380.381071,18.110803,3.875476,7.318627,35.270695,-0.177361,61.781565,8.684483,44.147513,2012.757081,0.724761,3.132878
std,1.996434e+17,12.221695,11.030274,1.092915,271.968318,9.745245,0.530663,4.667151,9.377598,0.706476,20.668727,3.609486,21.919804,3.979272,0.446641,13.368047
min,2.624300e+11,-1.000000,-1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,17.000000,-3.061979,0.000000,0.000000,-40.000000,2006.000000,0.000000,-73.000000
25%,2.931800e+11,3.000000,3.000000,2.000000,129.000000,10.000000,4.000000,4.000000,27.000000,-0.715004,49.000000,6.000000,30.000000,2009.000000,0.000000,-4.000000
50%,3.327800e+11,13.000000,10.000000,2.000000,372.000000,18.000000,4.000000,7.000000,35.000000,0.000000,68.000000,9.000000,48.000000,2013.000000,1.000000,3.000000
75%,4.008700e+17,23.000000,17.000000,3.000000,609.000000,25.000000,4.000000,10.000000,43.000000,0.000000,76.000000,11.000000,60.000000,2016.000000,1.000000,10.000000
max,4.011840e+17,80.000000,100.000000,8.000000,900.000000,100.000000,4.000000,82.000000,69.000000,3.409830,162.000000,174.000000,153.000000,2019.000000,1.000000,68.000000


In [0]:
#Create Histograms of all numeric data to check for skewness and kurtosis
def histograms(df, feature):
  from matplotlib import pyplot as plt
  import pandas as pd

  plt.hist(df[feature], bins=int(round(df[feature].count()**(1/3), 0)))

  text  = 'Std dev: ' + str(round(df[feature].std(), 2)) + '\n'
  text += 'Mean: ' + str(round(df[feature].mean(), 2)) + '\n'
  text += 'Skew: ' + str(round(df[feature].skew(), 2)) + '\n'
  text += 'Kurt: ' + str(round(df[feature].kurt(), 2))

  # plt.text(horizontal position, vertical position, text, ...)
  #          vertical and horizontal positions are percents on decimal form from 0 to 1
  #          vertical=0 starts at the far left; higher numbers push the text right
  #          horizontal=0 starts at the bottom; higher  numbers push the text up
  plt.text(0.35, 0.5, text, fontsize=10, transform=plt.gcf().transFigure)
  plt.axvline(df[feature].mean(), color='k', linestyle='dashed', linewidth=1)
  plt.title(feature)
  plt.xlabel(feature)
  plt.ylabel('count')
  plt.show()

import pandas as pd
df=pd.read_csv('https://jasonstewart.s3.amazonaws.com/college-football-data.csv')
for col in df:
  if pd.api.types.is_numeric_dtype(df[col]):
    histograms(df,col)

In [0]:
#Assign points for each drive_result
df=df[df['drive_result']!='KICKOFF']
df.drive_result.unique()

points_scored=[]
for row in range(len(df)):
  if df['drive_result'].iloc[row] in ['END OF 4TH QUARTER,''POSSESSION (FOR OT DRIVES)','DOWNS','DOWNS TD','FG TD','MISSED FG TD','INT','FUMBLE','TURNOVER ON DOWNS','INT TD','FUMBLE TD','FUMBLE RETURN TD','SF','FG MISSED','END OF HALF','END OF GAME','PUNT','Uncategorized','SF','PUNT RETURN TD','FUMBLE RETURN TD','FG MISSED TD','TURNOVER ON DOWNS TD','INT RETURN TOUCH','PUNT TD','MISSED FG',]:
    points_scored.append(0)
  elif df['drive_result'].iloc[row] in ['TD','PASSING TD','RUSHING TD','END OF HALF TD','END OF GAME TD','POSSESSION (FOR OT DRIVES) TD']:
    points_scored.append(6)
  elif df['drive_result'].iloc[row] in ['FG GOOD','FG GOOD TD','FG']:
    points_scored.append(3)
  else:
    points_scored.append(0)

df['points_scored']=points_scored

In [0]:
#Creates field for the NEXT_DRIVE which could potential later be used to see impact of momentum with either missed FG or failed 4th down conversion
# df.next_drive_drive_result.unique()
# next_drive_scoring=[]
# for row in range(len(df)):
#   if df['next_drive_drive_result'].iloc[row] in ['TD','RUSHING TD','PASSING TD']:
#     next_drive_scoring.append(-6)
#   elif df['next_drive_drive_result'].iloc[row] in ['INT','MISSED FG', 'PUNT','FUMBLE','END OF HALF','END OF GAME','TURNOVER ON DOWNS','DOWNS','FG MISSED','Uncategorized','TURNOVER ON DOWNS TD','PUNT TD','END OF 4TH QUARTER', 'POSSESSION (FOR OT DRIVES)']:
#     next_drive_scoring.append(0)
#   elif df['next_drive_drive_result'].iloc[row] in ['FG']:
#     next_drive_scoring.append(-3)
#   elif df['next_drive_drive_result'].iloc[row] in ['SF']:
#     next_drive_scoring.append(2) 
#   else:
#     next_drive_scoring.append(0)
# df['next_drive_points_scored']=next_drive_scoring

In [16]:
#Creating the fg_length field correctly, pulling from play_text
count=0
for item in df['play_text']:
  array = item.split(' ')
  #print(array)
  for item in array:
    if item.isdigit():
      df['fg_length'].iloc[count]=int(item)
      break
  count+=1
df['fg_length']=df['fg_length'].astype(int)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [17]:
#puts the correct kicker_name
count=0
for item in df['play_text']:
  array = item.split(' ')
  countitems=0
  for item in array:
    if item.isdigit():
      kicker_name=''
      for num in range(countitems):
        kicker_name+=array[num]+' '
      df['kicker_name'].iloc[count]=kicker_name[:-1]
      break
    countitems+=1
  count+=1


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
#Creating a fg percentage for each kicker by going through all the kicks for each kicker and calculating percentages

fiftyplus=[] #percentages
fifty=[] #percentages
fourty=[] #percentages
thirty=[] #percentages
twenty=[] #percentages
fg_percentage=[] #percentages
fiftyplusattempts=[]


for item in df.kicker_name:
  df_kicker=df.filter(like=item, axis=0)
  for num in range(len(df_kicker)):
    count20=[0,0]
    count30=[0,0]
    count40=[0,0]
    count50=[0,0]
    count50plus=[0,0]
    if df_kicker['fg_length'].iloc[num]<20:
      if df_kicker['fg_result'].iloc[num]==1:
        count20[0]=count20[0]+1
      count20[1]=count20[1]+1
    elif df_kicker['fg_length'].iloc[num]<30:
      if df_kicker['fg_result'].iloc[num]==1:
        count30[0]=count30[0]+1
      count30[1]=count30[1]+1

    elif df_kicker['fg_length'].iloc[num]<40:
      if df_kicker['fg_result'].iloc[num]==1:
        count40[0]=count40[0]+1
      count40[1]=count40[1]+1

    elif df_kicker['fg_length'].iloc[num]<50:
      if df_kicker['fg_result'].iloc[num]==1:
        count50[0]=count50[0]+1
      count50[1]=count50[1]+1

    else:
      if df_kicker['fg_result'].iloc[num]==1:
        count50plus[0]=count50plus[0]+1
      count50plus[1]=count50plus[1]+1

    fiftyplusattempts=count50plus[1]
    fiftyplus.append(float(count50plus[0]/count50plus[1]))
    fifty.append(float(count50[0]/count50[1]))
    fourty.append(float(count40[0]/count40[1]))
    thirty.append(float(count30[0]/count30[1]))
    twenty.append(float(count20[0]/count20[1]))
    attempts=count20[1]+count30[1]+count40[1]+count50[1]+count50plus[1]
    makes=count20[0]+count30[0]+count40[0]+count50[0]+count50plus[0]
    fg_percentage.append(float(makes/attempts))



In [19]:
#binary result of each fg kick
results=[]
field_goal_length=[]
for item in df['play_type']:
  if item=='Field Goal Good':
    results.append(1)
  else:
    results.append(0)
count=0
df['fg_result']=results
df

,id,offense,offense_conference,defense,defense_conference,home,away,offense_score,defense_score,period,clock,yards_to_goal,down,distance,kicker_name,play_type,play_text,fg_length,ppa,drive_start_yard_line,number_of_plays_in_drive,yards_gained_during_drive,drive_result,season,fg_result,score_diff,points_scored
0,3.130900e+11,Iowa State,Big 12,Kansas,Big 12,Iowa State ...,Kansas,3,7,2,810,68,1,10,- 1a,Field Goal Missed,- 1a 49 yard field goal BLOCKED.,49,0.000000,68,1,0,FG MISSED,2011,0,-4,0
1,2.727930e+11,Western Michigan,Mid-American,Akron,Mid-American,Western Michigan ...,Akron,3,3,1,800,66,1,10,-,Field Goal Good,- 45 51 yard field goal GOOD.,45,1.336769,66,9,65,FG GOOD,2007,1,0,3
2,2.824100e+11,South Dakota State,None,Iowa State,Big 12,Iowa State ...,South Dakota State,3,20,3,540,19,4,4,- Peterreifenrath,Field Goal Good,- Peterreifenrath 35 yard field goal GOOD.,35,-0.433847,75,12,56,FG GOOD,2008,1,-17,3
3,4.007650e+17,Cincinnati,American Athletic,South Florida,American Athletic,South Florida ...,Cincinnati,3,34,2,557,27,4,8,A. Gantz,Field Goal Good,A. Gantz 44 yd FG GOOD,44,0.599430,78,12,51,FG,2015,1,-31,3
4,3.025400e+11,New Hampshire,None,Pittsburgh,Big East,Pittsburgh ...,New Hampshire,3,10,2,714,5,4,5,A.J. DeLago,Field Goal Good,A.J. DeLago 22 yard field goal GOOD.,22,-0.947408,80,10,75,FG GOOD,2010,1,-7,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32792,3.225900e+11,North Dakota,None,San Diego State,Mountain West,San Diego State ...,North Dakota,20,35,2,32,26,4,12,Zeb Miller,Field Goal Good,Zeb Miller 43 yard field goal GOOD.,43,0.968056,75,9,49,FG GOOD,2012,1,-15,3
32793,3.125320e+11,Grambling,None,Louisiana Monroe,Sun Belt,Louisiana Monroe ...,Grambling,7,14,2,468,28,4,16,Zoltan Riazzo,Field Goal Missed,Zoltan Riazzo 45 yard field goal MISSED.,45,0.000000,47,6,19,FG MISSED,2011,0,-7,0
32794,3.024730e+11,Grambling,None,Louisiana Tech,Western Athletic,Grambling ...,Louisiana Tech,3,17,2,8,35,4,12,Zoltan Riazzo,Field Goal Missed,Zoltan Riazzo 52 yard field goal MISSED.,52,0.000000,44,5,9,FG MISSED,2010,0,-14,0
32795,3.024730e+11,Grambling,None,Louisiana Tech,Western Athletic,Grambling ...,Louisiana Tech,3,7,1,528,22,4,10,Zoltan Riazzo,Field Goal Good,Zoltan Riazzo 39 yard field goal GOOD.,39,0.366953,80,8,58,FG GOOD,2010,1,-4,3


In [20]:
#calculates correctly the fg_length, basically checking to make sure it is an int
fgl=[]
for num in range(len(df)):
  row=df.iloc[num]
  if row.play_type=='Blocked Field Goal Touchdown' or row.play_type=='Missed Field Goal Return Touchdown':
    df.drop(df.iloc[num])
  else:
    if not isinstance(int(row.fg_length), int):
      df.drop(df.iloc[num])
    else:
      row.fg_length=int(row.fg_length)
df['fg_length'].unique()


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


array([49, 45, 35, 44, 22, 42, 23, 36, 43, 40, 24, 28, 32, 27, 38, 33, 30,
       20, 41, 31, 46, 34, 47, 21, 39, 26, 19, 25, 29, 52, 48, 51, 50, 37,
       58, 54, 55, 18, 53, 56, 57, 17, 61, 60, 59, 67, 69, 62, 64, 66, 65,
       63])

In [21]:
#Creating a categorical way to establish the home and away team, home teams are described using the number 1
home=[]
for num in range(len(df)):
  #print(df['offense'].iloc[num],len(df['offense'].iloc[num]),df['home'].iloc[num],len(df['home'].iloc[num]))
  hometeam=df['home'].iloc[num]
  if df['offense'].iloc[num]==hometeam [:len(df['offense'].iloc[num])]:
    home.append(1)
  else:
    home.append(0)
df['home']=home
df.drop('away',axis=1, inplace=True)
df
# kicker_data=[]
# for row in range(len(df)):
#   df['kicker_name'],

,id,offense,offense_conference,defense,defense_conference,home,offense_score,defense_score,period,clock,yards_to_goal,down,distance,kicker_name,play_type,play_text,fg_length,ppa,drive_start_yard_line,number_of_plays_in_drive,yards_gained_during_drive,drive_result,season,fg_result,score_diff,points_scored
0,3.130900e+11,Iowa State,Big 12,Kansas,Big 12,1,3,7,2,810,68,1,10,- 1a,Field Goal Missed,- 1a 49 yard field goal BLOCKED.,49,0.000000,68,1,0,FG MISSED,2011,0,-4,0
1,2.727930e+11,Western Michigan,Mid-American,Akron,Mid-American,1,3,3,1,800,66,1,10,-,Field Goal Good,- 45 51 yard field goal GOOD.,45,1.336769,66,9,65,FG GOOD,2007,1,0,3
2,2.824100e+11,South Dakota State,None,Iowa State,Big 12,0,3,20,3,540,19,4,4,- Peterreifenrath,Field Goal Good,- Peterreifenrath 35 yard field goal GOOD.,35,-0.433847,75,12,56,FG GOOD,2008,1,-17,3
3,4.007650e+17,Cincinnati,American Athletic,South Florida,American Athletic,0,3,34,2,557,27,4,8,A. Gantz,Field Goal Good,A. Gantz 44 yd FG GOOD,44,0.599430,78,12,51,FG,2015,1,-31,3
4,3.025400e+11,New Hampshire,None,Pittsburgh,Big East,0,3,10,2,714,5,4,5,A.J. DeLago,Field Goal Good,A.J. DeLago 22 yard field goal GOOD.,22,-0.947408,80,10,75,FG GOOD,2010,1,-7,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32792,3.225900e+11,North Dakota,None,San Diego State,Mountain West,0,20,35,2,32,26,4,12,Zeb Miller,Field Goal Good,Zeb Miller 43 yard field goal GOOD.,43,0.968056,75,9,49,FG GOOD,2012,1,-15,3
32793,3.125320e+11,Grambling,None,Louisiana Monroe,Sun Belt,0,7,14,2,468,28,4,16,Zoltan Riazzo,Field Goal Missed,Zoltan Riazzo 45 yard field goal MISSED.,45,0.000000,47,6,19,FG MISSED,2011,0,-7,0
32794,3.024730e+11,Grambling,None,Louisiana Tech,Western Athletic,1,3,17,2,8,35,4,12,Zoltan Riazzo,Field Goal Missed,Zoltan Riazzo 52 yard field goal MISSED.,52,0.000000,44,5,9,FG MISSED,2010,0,-14,0
32795,3.024730e+11,Grambling,None,Louisiana Tech,Western Athletic,1,3,7,1,528,22,4,10,Zoltan Riazzo,Field Goal Good,Zoltan Riazzo 39 yard field goal GOOD.,39,0.366953,80,8,58,FG GOOD,2010,1,-4,3


In [0]:
#FIX SKEWNESS

def fix_skew(df, feature):
  import numpy as np
  # check for positive skewness
  if df[feature].skew() > 1:
    # generate the new columns to test
    df[feature + '_SR'] = np.sqrt(df[feature])
    df[feature + '_CR'] = np.cbrt(df[feature])
    df[feature + '_LN'] = np.log(df[feature] + 1)
    # df[feature + '_BIN'] = bin_numeric(df[feature])[2]
    print(df.skew())
    
    # get dummies
    for col in df:
      if not pd.api.types.is_numeric_dtype(df[col]):
        df = df.join(pd.get_dummies(df[col], prefix=col, drop_first=True))

    # get numeric columns
    df = df.select_dtypes(np.number)
    df_minmax = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df), columns=df.columns)

    # test original feature
    y = df_minmax[feature]
    X = df_minmax.drop(columns=[feature, feature + '_LN', feature + '_SR', feature + '_CR']).assign(const=1)

    results = sm.OLS(y, X).fit()
    print(results.summary())

    # test square root feature
    y = df_minmax[feature + '_SR']
    X = df_minmax.drop(columns=[feature, feature + '_LN', feature + '_SR', feature + '_CR']).assign(const=1)

    results = sm.OLS(y, X).fit()
    print(results.summary())

    # test cube root feature
    y = df_minmax[feature + '_CR']
    X = df_minmax.drop(columns=[feature, feature + '_LN', feature + '_SR', feature + '_CR']).assign(const=1)

    results = sm.OLS(y, X).fit()
    print(results.summary())

    # test log feature
    y = df_minmax[feature + '_LN']
    X = df_minmax.drop(columns=[feature, feature + '_LN', feature + '_SR', feature + '_CR']).assign(const=1)

    results = sm.OLS(y, X).fit()
    print(results.summary())

    # test binning feature
    # y = df_minmax[feature + '_BIN']
    # X = df_minmax.drop(columns=[feature, feature + '_LN', feature + '_SR', feature + '_CR', feature + '_BIN']).assign(const=1)

    # results = sm.OLS(y, X).fit()
    # print(results.summary())


  elif df[feature].skew() < -1:
    # here we would fix negatively skewed variables if there were any in this dataset

    return
  else:
    # return if skewness is between -1 and 1; negligible
    return

In [0]:
#CORRECT OUTLIERS, this first part checks to see if there are outliers
outliers = pd.DataFrame(columns=['min', 'count below', 'max', 'count above'])

# save the max to compare later
# peak_max = df[''].mean() + (df[''].std() * 3)

# Loop through each column in the dataframe except those that are: 1) categorical, 2) empty, or 3) binary (0/1)
for col in df:
  if pd.api.types.is_numeric_dtype(df[col]) and (len(df[col].value_counts()) > 0) and not all(df[col].value_counts().index.isin([0, 1])):
    # Calculate the theoretical min/max
    min = df[col].mean() - (df[col].std() * 3)
    max = df[col].mean() + (df[col].std() * 3)
    outliers.loc[col] = (min, df[col][df[col] < min].count(), max, df[col][df[col] > max].count())

outliers

In [0]:
#Goes through each column in DF to replace outliers with min or max
def manage_outliers(df):
  for col in df:
    if pd.api.types.is_numeric_dtype(df[col]) and (len(df[col].value_counts()) > 0) and not all(df[col].value_counts().index.isin([0, 1])):
      min = df[col].mean() - (df[col].std() * 3)
      max = df[col].mean() + (df[col].std() * 3)

      # Once the min/max is calculated for a column, take an inner loop through the row values of each column to replace values
      for i, value in enumerate(df[col]): 
        if value < min:      
          df.at[i, col + '_3std'] = min  # replace with min
          df.at[i, 'outlier'] = 1         # identify the record as an outlier for future filtering
        elif value > max:    
          df.at[i, col + '_3std'] = max   # replace with max
          df.at[i, 'outlier'] = 1
        else:                
          df.at[i, col + '_3std'] = value # use original value
          df.at[i, 'outlier'] = 0
    
  # I don't like binary variables like "outlier" becoming floats; cast it back to an int
  # if 'outlier' in df.columns:
  #   df = df.astype({"outlier": int})

  # View the cleaned dataset:
  print(df.head())

In [0]:
#There shouldnt be any missing data, like I am 99% sure there isnt missing data but could be good to double check

#THIS BELOW COMES STRAIGHT FROM THE BOOK BUT IT SHOULD WORK
def manage_missing_data(df):
      import pandas as pd
      from scipy import stats

      for col_y in df:                                        # for each column in df to fill
        if df[col_y].isnull().sum() > 0:                      # if there is missing data to fill
          if is_numeric(df[col_y]):                           # and if that column is numeric
            r_list = df.corr()[col_y].abs()                   # create a correlation matrix for just the col_y feature; get absolute values to eliminate sign (+/-)
            r_list.sort_values(inplace=True, ascending=False) # sort the list descending (we want the second item in the list because the first is itself where r = 1.0)

            col_x = r_list.index[1] # Store the index name of the highest correlation

            # Create a new df with missing values of y and x dropped
            clean_df = df.copy()
            clean_df.dropna(subset=[col_y, col_x], inplace=True)

            # Calculate the regression line: y = m(x) + b
            m, b, r, p, err = stats.linregress(clean_df[col_x], clean_df[col_y])

            # enumerate() allows us to include a count-based index (i) along with the value
            for i, value in enumerate(df[col_y].values):
              if not pd.notnull(value):
                imputed_value = round((m * df[col_x][i]) + b, 1)

                # if the prediction is outside the range of actual values, then replace it with the min/max
                if imputed_value < df[col_y].min():
                  df[col_y][i] = df[col_y].min()
                elif imputed_value > df[col_y].max():
                  df[col_y][i] = df[col_y].max()
                else:
                  df[col_y][i] = imputed_value
      return df

In [0]:
def data_cleaning(df):
  manage_missing_data(df)
  manage_outliers(df)
  for col in df:
    fix_skew(df, col)
  

In [0]:
df.to_csv('college-football-data-p5-fg-final.csv')

In [0]:
df